In [3]:
import pandas as pd
import numpy as np
# 假设历史案例数据如下
data = [
    {'budget': 5000, 'purpose': 'gaming', 'cpu': 'i5-11400F', 'gpu': 'RTX 3060', 'ram': '16G', 'ssd': '512G'},
    {'budget': 6000, 'purpose': 'gaming', 'cpu': 'i5-12600KF', 'gpu': 'RTX 3060Ti', 'ram': '16G', 'ssd': '512G'},
    {'budget': 8000, 'purpose': 'gaming', 'cpu': 'i7-12700F', 'gpu': 'RTX 3070', 'ram': '32G', 'ssd': '1T'},
    {'budget': 4000, 'purpose': 'office', 'cpu': 'i5-12400F', 'gpu': 'RTX 3050', 'ram': '16G', 'ssd': '256G'},
    {'budget': 3000, 'purpose': 'office', 'cpu': 'i3-12100F', 'gpu': 'GTX 1650', 'ram': '8G', 'ssd': '256G'},
]

df = pd.DataFrame(data)

In [8]:
from sklearn.preprocessing import OneHotEncoder

# 用途转 one-hot 编码
enc = OneHotEncoder(handle_unknown='ignore')
purpose_encoded = enc.fit_transform(df[['purpose']]).toarray()

# 预算归一化到 0-1 区间
budget_min, budget_max = df['budget'].min(), df['budget'].max()
budget_normalized = (df['budget'] - budget_min) / (budget_max - budget_min)

# 组装特征矩阵
X = np.hstack([budget_normalized.values.reshape(-1, 1), purpose_encoded])  # 修改了这一行

from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

y = df[['cpu', 'gpu', 'ram', 'ssd']]

model = MultiOutputClassifier(RandomForestClassifier())
model.fit(X, y)

# 假设新需求如下
new_req = {'budget': 7000, 'purpose': 'gaming'}

# 特征转换 
new_budget_normalized = (new_req['budget'] - budget_min) / (budget_max - budget_min)
new_purpose_encoded = enc.transform([[new_req['purpose']]]).toarray()
new_X = np.hstack([new_budget_normalized.reshape(1, -1), new_purpose_encoded])  # 修改了这一行

# 用模型预测
rec = model.predict(new_X)
print(dict(zip(['cpu', 'gpu', 'ram', 'ssd'], rec[0])))

{'cpu': 'i5-12600KF', 'gpu': 'RTX 3060Ti', 'ram': '16G', 'ssd': '512G'}


/Users/czl/code/cuan/ml/recommend/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
